In [1]:
import pandas as pd
import numpy as np
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pickle
import os
import plotly.express as px
import base64
from scipy.interpolate import interp1d
import re
import collections
import socket
import struct
import time

In [14]:
ScalerPath = "./Model/Scaler.pickle"
AccelRNNModelPath = "./Model/AccelRNNModel.pickle"
CORNNModelPath = "./Model/CORNNModel.pickle"

In [15]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, seq_length, num_layers):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.seq_length = seq_length


        # GRU layer
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size * seq_length, output_size)

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Reshape output to fit the fully connected layer
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        print("Shape of output before FC:", out.shape)
        return out

In [16]:
scaler=None
if os.path.exists(ScalerPath):
    with open(ScalerPath, 'rb') as file:
        scaler = pickle.load(file)
else:
    print ("Could NOT find Scaler")

In [17]:
inputHeader = ["ApproachRateOther", 
               "Rel_Pos_Magnitude", 
               "SteerB", 
               "A_Head_Center_Distance", "B_Head_Center_Distance", 
               "Filtered_B_Head_Velocity_Total",
               "A_Turn",
               "B_Indicator",
               "Centerline_Offset_B",
               "RelativeRotation"]

# Aceeleration RNN Model

In [18]:
HeaderWithoutAccel = ["ApproachRateOther", 
                      "Rel_Pos_Magnitude", 
                      "SteerB", 
                      "A_Head_Center_Distance", "B_Head_Center_Distance", 
                      "Filtered_B_Head_Velocity_Total",
                      "A_Turn",
                      "B_Indicator",
                      "Centerline_Offset_B",
                      "RelativeRotation"]

outputHeader = ["AccelA"]

input_size = len(inputHeader)
output_size = len(outputHeader)

Seq_Length = 13
hidden_size = 64
num_layers = 1

accelA_net = GRUModel(input_size=input_size, 
                      hidden_size=hidden_size, 
                      seq_length=Seq_Length,
                      output_size=output_size, 
                      num_layers=num_layers)
    
accelA_net.load_state_dict(torch.load(AccelRNNModelPath))
accelA_net.eval()

GRUModel(
  (rnn): GRU(10, 64, batch_first=True)
  (fc): Linear(in_features=832, out_features=1, bias=True)
)

# Centerline Offset RNN Model

In [19]:
outputHeader = ["Centerline_Offset_A"]

input_size = len(inputHeader)
output_size = len(outputHeader)

Seq_Length = 13
hidden_size = 64
num_layers = 3

coA_net = GRUModel(input_size=input_size, 
                   hidden_size=hidden_size, 
                   seq_length=Seq_Length,
                   output_size=output_size, 
                   num_layers=num_layers)
    
coA_net.load_state_dict(torch.load(CORNNModelPath))
coA_net.eval()

GRUModel(
  (rnn): GRU(10, 64, num_layers=3, batch_first=True)
  (fc): Linear(in_features=832, out_features=1, bias=True)
)

# Initial Status

In [20]:
d = collections.deque(maxlen=Seq_Length)

for i in range(Seq_Length):
    d.append(np.ones(input_size)+i)
scaled_features = scaler.transform(np.array(d))

In [21]:
accelA_net(torch.Tensor([scaled_features])).item()

Shape of output before FC: torch.Size([1, 1])


0.43574059009552

In [22]:
coA_net(torch.Tensor([scaled_features])).item()

Shape of output before FC: torch.Size([1, 1])


0.15810000896453857

# Data Socket


In [23]:
UDP_IP = "127.0.0.1"
txPort = 8000
rxPort =8001
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.bind((UDP_IP, rxPort))

In [24]:
while True:
    try:
        data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
       
        if(int(len(data)/4)!=10):
            print("Didnt get enoughb bytes, continuing!",int(len(data)/4))
            continue
        array = struct.unpack('ffffffffff', data) 

        d.append(np.array(array))
        scaled_features = scaler.transform(np.array(d))
        
        # val = net(torch.Tensor([scaled_features]))
        # Predict using accelA_net
        accelA_val = accelA_net(torch.Tensor([scaled_features])).item()
        # Predict using coA_net
        coA_val = coA_net(torch.Tensor([scaled_features])).item()

        # Send predictions
        # floatlist=[val]
        floatlist = [accelA_val, coA_val]

        #print(array,val)
        buf = struct.pack('%sf' % len(floatlist), *floatlist)
        sock.sendto(buf, (UDP_IP, txPort))
    except BlockingIOError:
        time.sleep(1/19)

KeyboardInterrupt: 